In [2]:
from qiskit import *
from qiskit.visualization import *
from qiskit.transpiler import PassManager
#from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.providers.ibmq import least_busy
from qiskit.aqua.algorithms import ExactEigensolver
from qiskit.aqua.algorithms import QPE
from qiskit.aqua import QuantumInstance
from qiskit.aqua.components.initial_states import Custom
from qiskit.aqua import Operator
from qiskit.aqua.components.iqfts import Standard
import numpy as np
import math

In [3]:
IBMQ.load_accounts()

In [4]:
def matrixHamiltonian(tam, freq): #tam 4, estados 0 a 3
    h = 6.582119624 * 10**(-16)
    E = [h*freq*(i+1/2) for i in range(tam)]
    L = []
    for i in range(tam):
        L.append(list())
        for j in range(tam):
            if i==j: L[i].append(E[i])
            else: L[i].append(0)
    m = np.matrix(L)
    return m


In [9]:
m.item(1,1)

9.873179436

In [ ]:
m = matrixHamiltonian(4, 10**16)
op = Operator(matrix=m)
eig_res = ExactEigensolver(op, k=4)
eig_res = eig_res.run()
eigv = eig_res['eigvecs']

state = [Custom(2, state_vector=eigv[i]) for i in range(4)]

num_time_slices = 50
n_ancillae = 9
iqft = Standard(n_ancillae)
results=[]
for i in range(4):
    qpe = QPE(op, state[i], iqft, num_time_slices, n_ancillae, expansion_mode='suzuki',
              expansion_order=2, shallow_circuit_concat=True)

    backend = BasicAer.get_backend('qasm_simulator')
    quantum_instance = QuantumInstance(backend, shots=100, pass_manager=PassManager())
    result_qpe = qpe.run(quantum_instance)
    results.append(result_qpe['energy'])
print(results)

In [35]:
result_qpe['energy']

3.3296269191718757

In [41]:
Custom?